# TorchSDE + Neuromancer: Latent Stochastic Differential Equations (System ID of Stochastic Process)

This notebook goes over how to utilize torchsde's functionality within Neuromancer framework. This notebook is based off: https://github.com/google-research/torchsde/blob/master/examples/latent_sde_lorenz.py. In this example, we generate data according to a 3-dimensional stochastic Lorenz attractor. We then perform a "system identification" on this data -- seek to model a stochastic differential equation on this data. Upon performant training, this LatentSDE will then be able to reproduce samples that exhibit the same behavior as the provided Lorenz system. 




### Imports

In [1]:
import torch
from neuromancer.psl import plot
from neuromancer import psl
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from neuromancer.system import Node
from neuromancer.dynamics import integrators, ode
from neuromancer.trainer import Trainer
from neuromancer.problem import Problem
from neuromancer.loggers import BasicLogger
from neuromancer.dataset import DictDataset
from neuromancer.constraint import variable
from neuromancer.loss import PenaltyLoss
from neuromancer.modules import blocks

torch.manual_seed(0)


In [2]:
import logging
import os
from typing import Sequence

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import torch
import tqdm
from torch import nn
from torch import optim
from torch.distributions import Normal

import torchsde

### Functions to generate data from a Lorenz attractor

In [3]:
class LinearScheduler(object):
    def __init__(self, iters, maxval=1.0):
        self._iters = max(1, iters)
        self._val = maxval / self._iters
        self._maxval = maxval

    def step(self):
        self._val = min(self._maxval, self._val + self._maxval / self._iters)

    @property
    def val(self):
        return self._val


class StochasticLorenz(object):
    """Stochastic Lorenz attractor.

    Used for simulating ground truth and obtaining noisy data.
    Details described in Section 7.2 https://arxiv.org/pdf/2001.01328.pdf
    Default a, b from https://openreview.net/pdf?id=HkzRQhR9YX
    """
    noise_type = "diagonal"
    sde_type = "ito"

    def __init__(self, a: Sequence = (10., 28., 8 / 3), b: Sequence = (.1, .28, .3)):
        super(StochasticLorenz, self).__init__()
        self.a = a
        self.b = b

    def f(self, t, y):
        x1, x2, x3 = torch.split(y, split_size_or_sections=(1, 1, 1), dim=1)
        a1, a2, a3 = self.a

        f1 = a1 * (x2 - x1)
        f2 = a2 * x1 - x2 - x1 * x3
        f3 = x1 * x2 - a3 * x3
        return torch.cat([f1, f2, f3], dim=1)

    def g(self, t, y):
        x1, x2, x3 = torch.split(y, split_size_or_sections=(1, 1, 1), dim=1)
        b1, b2, b3 = self.b

        g1 = x1 * b1
        g2 = x2 * b2
        g3 = x3 * b3
        return torch.cat([g1, g2, g3], dim=1)

    @torch.no_grad()
    def sample(self, x0, ts, noise_std, normalize):
        """Sample data for training. Store data normalization constants if necessary."""
        xs = torchsde.sdeint(self, x0, ts)
        if normalize:
            mean, std = torch.mean(xs, dim=(0, 1)), torch.std(xs, dim=(0, 1))
            xs.sub_(mean).div_(std).add_(torch.randn_like(xs) * noise_std)
        return xs




def vis(xs, ts, latent_sde, bm_vis, img_path, num_samples=10):
    fig = plt.figure(figsize=(20, 9))
    gs = gridspec.GridSpec(1, 2)
    ax00 = fig.add_subplot(gs[0, 0], projection='3d')
    ax01 = fig.add_subplot(gs[0, 1], projection='3d')

    # Left plot: data.
    z1, z2, z3 = np.split(xs.cpu().numpy(), indices_or_sections=3, axis=-1)
    [ax00.plot(z1[:, i, 0], z2[:, i, 0], z3[:, i, 0]) for i in range(num_samples)]
    ax00.scatter(z1[0, :num_samples, 0], z2[0, :num_samples, 0], z3[0, :10, 0], marker='x')
    ax00.set_yticklabels([])
    ax00.set_xticklabels([])
    ax00.set_zticklabels([])
    ax00.set_xlabel('$z_1$', labelpad=0., fontsize=16)
    ax00.set_ylabel('$z_2$', labelpad=.5, fontsize=16)
    ax00.set_zlabel('$z_3$', labelpad=0., horizontalalignment='center', fontsize=16)
    ax00.set_title('Data', fontsize=20)
    xlim = ax00.get_xlim()
    ylim = ax00.get_ylim()
    zlim = ax00.get_zlim()

    # Right plot: samples from learned model.
    xs = latent_sde.sample(batch_size=xs.size(1), ts=ts, bm=bm_vis).cpu().numpy()
    z1, z2, z3 = np.split(xs, indices_or_sections=3, axis=-1)

    [ax01.plot(z1[:, i, 0], z2[:, i, 0], z3[:, i, 0]) for i in range(num_samples)]
    ax01.scatter(z1[0, :num_samples, 0], z2[0, :num_samples, 0], z3[0, :10, 0], marker='x')
    ax01.set_yticklabels([])
    ax01.set_xticklabels([])
    ax01.set_zticklabels([])
    ax01.set_xlabel('$z_1$', labelpad=0., fontsize=16)
    ax01.set_ylabel('$z_2$', labelpad=.5, fontsize=16)
    ax01.set_zlabel('$z_3$', labelpad=0., horizontalalignment='center', fontsize=16)
    ax01.set_title('Samples', fontsize=20)
    ax01.set_xlim(xlim)
    ax01.set_ylim(ylim)
    ax01.set_zlim(zlim)

    plt.savefig(img_path)
    plt.close()

In [22]:
batch_size=1024
latent_size=4
context_size=64
hidden_size=128
lr_init=1e-2
t0=0.
t1=2.
lr_gamma=0.997
num_iters=1
kl_anneal_iters=1000
pause_every=50
noise_std=0.01
adjoint=False
train_dir='./dump/lorenz/'
method="euler"

def make_dataset(t0, t1, batch_size, noise_std, train_dir, device):
    data_path = os.path.join(train_dir, 'lorenz_data.pth')
    
    _y0 = torch.randn(batch_size, 3, device=device)
    ts = torch.linspace(t0, t1, steps=100, device=device)
    xs = StochasticLorenz().sample(_y0, ts, noise_std, normalize=True)

    os.makedirs(os.path.dirname(data_path), exist_ok=True)
    torch.save({'xs': xs, 'ts': ts}, data_path)
    logging.warning(f'Stored toy data at: {data_path}')
    return xs, ts


## Neuromancer Integration

Generate the data and create Neuromancer DictDataset

In [29]:

device='cpu'
torch.manual_seed(0)
xs, ts = make_dataset(t0=t0, t1=t1, batch_size=batch_size, noise_std=noise_std, train_dir=train_dir, device=device)
train_data = DictDataset({'xs':xs},name='train')
train_data_loader = DataLoader(train_data, batch_size=1024, collate_fn=train_data.collate_fn, shuffle=False)


In [32]:
xs.shape

torch.Size([100, 1, 3])

In [31]:
baz = next(iter(train_data_loader))
baz['xs'].shape

torch.Size([100, 1, 3])

Define Neuromancer components, variables, and problem to train the LatentSDE. Upon training, this LatentSDE will generate new samples that exhibit the behavior of the Lorenz attractor training data

In [17]:
sde_block_encoder = blocks.LatentSDE_Encoder(3, latent_size, context_size, hidden_size, ts=ts) 
integrator = integrators.LatentSDEIntegrator(sde_block_encoder)
model_1 = Node(integrator, input_keys=['xs'], output_keys=['zs', 'z0', 'log_ratio',  'xs', 'qz0_mean', 'qz0_logstd'], name='m1')
sde_block_decoder = blocks.LatentSDE_Decoder(3, latent_size, noise_std=noise_std)
model_2 = Node(sde_block_decoder, input_keys=['xs', 'zs', 'log_ratio', 'qz0_mean', 'qz0_logstd'], output_keys=['log_pxs', 'sum_term', 'log_ratio'], name='m2' )

xs = variable('xs')
zs = variable('zs')
z0 = variable('z0')


log_ratio = variable('log_ratio')
qz0_mean = variable('qz0_mean')
qz0_logstd = variable('qz0_logstd')
log_pxs = variable('log_pxs')
sum_term = variable('sum_term')



loss = (-1.0*log_pxs + log_ratio) == 0.0


# aggregate list of objective terms and constraints
objectives = [loss]
constraints = []
# create constrained optimization loss
loss = PenaltyLoss(objectives, constraints)
# construct constrained optimization problem
problem = Problem([model_1, model_2], loss)

### Neuromancer training the problem to learn the stochastic process

In [18]:
optimizer = torch.optim.Adam(problem.parameters(), lr=0.001)


# define neuromancer trainer
trainer = Trainer(
    problem,
    train_data_loader,
    train_data_loader,
    train_data_loader,
    optimizer,
    patience=0,
    warmup=0,
    epochs=15,
    eval_metric="train_loss",
    train_metric="train_loss",
    dev_metric="train_loss",
    test_metric="train_loss"
)
trainer.train()

/home/birm560/neuromancer/src/neuromancer/constraint.py:160: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([99, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.l1_loss(left, right)


epoch: 0  train_loss: 1733993.375
epoch: 1  train_loss: 1752146.75
Early stopping!!!


OrderedDict([('nodes.0.callable.block.pz0_mean', tensor([[0., 0., 0., 0.]])),
             ('nodes.0.callable.block.pz0_logstd', tensor([[0., 0., 0., 0.]])),
             ('nodes.0.callable.block.encoder.gru.weight_ih_l0',
              tensor([[ 0.0622,  0.0806,  0.0510],
                      [ 0.0128,  0.0423, -0.0440],
                      [-0.0742, -0.0770,  0.0892],
                      ...,
                      [ 0.0462,  0.0818, -0.0287],
                      [ 0.0035,  0.0163,  0.0892],
                      [ 0.0800, -0.0467, -0.0742]])),
             ('nodes.0.callable.block.encoder.gru.weight_hh_l0',
              tensor([[-0.0787, -0.0416,  0.0753,  ...,  0.0738,  0.0458, -0.0140],
                      [ 0.0055, -0.0083,  0.0195,  ..., -0.0700,  0.0129,  0.0738],
                      [ 0.0586,  0.0747,  0.0488,  ...,  0.0444,  0.0043,  0.0752],
                      ...,
                      [ 0.0160, -0.0344, -0.0338,  ..., -0.0417, -0.0702, -0.0172],
             

### Visualization of Learned Stochastic Samples

*TODO*